# **Árvore de Regressão e Random Forest**

* Parte 1: Entendimento e divisão do dataset
* Parte 2: Aplicação do algoritmo DecisionTreeRegressor
* Parte 3: Avaliação de desempenho
* Parte 4: Aplicação do algoritmo RandomForest
* Parte 5: Avaliação de desempenho

In [2]:
#Bibliotecas necessárias, modelos e métricas do scikit-learn e outros
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler, MinMaxScaler, KernelCenterer,Normalizer

from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,\
                                    validation_curve, cross_val_score, \
                                    cross_validate, \
                                    RepeatedKFold, cross_val_predict, StratifiedKFold
from scipy.stats import uniform, randint


# <font color=white>**Parte 1**: Entendimento e divisão do dataset

Definição das variáveis
- so2 IAG
- so2_1 estação
- so2_2 estação
- so2_3 estação
- so2_4 estação
- so2_we (working electord)
- so2_ae (auxiliary electrodo)
- best_so

In [3]:
#Leitura da base de dados da estação grafana com os dados de CO
df = pd.read_csv('Best_CO-data_06-25.csv')
#df1 = pd.read_csv('Best_NO2_06-25.csv')
#df2 = pd.read_csv('Best_SO2-data_06-25.csv')
df = df.drop(columns = ['time'])
df.head()

,CO
0,815.469921
1,815.479525
2,815.589499
3,815.680502
4,815.482320


In [4]:
#Leitura da base de dados da estação IAG de São Paulo
#print(aqm.shape)
#print(aqm.describe())
aqm = pd.read_csv('envcity_aqm_df.csv')
aqm.head()

,time,e1_anem,e1_anem_volt,e1_co,e1_co_2,e1_co_3,e1_co_4,e1_co_ae,e1_co_we,e1_location,...,iag_pm25,iag_pmcoarse,iag_pressure,iag_rainfalltotal,iag_sigma,iag_so2,iag_solarradiation,iag_wd,iag_ws,pin_umid
0,2023-02-15 13:41:00,12.036327,NaN,-1315.716919,459.329041,465.226715,598.345581,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-15 13:42:00,11.174897,NaN,-1575.367554,385.721252,395.833496,624.081055,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-15 13:43:00,11.703624,NaN,-3228.860352,-520.005371,-492.953430,117.647057,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-15 13:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-15 13:45:00,11.888990,NaN,-1663.602783,315.461090,325.980530,563.419250,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#filtrando as linhas onde e1_co tem valores válidos
df_filtrado = aqm[aqm['e1_co'].notna()][['time', 'e1_co', 'iag_co']]

#remover linhas onde iag_co é NaN
df_filtradoIAG = df_filtrado[df_filtrado['iag_co'].notna()]

df_filtradoIAG 

,time,e1_co,iag_co
40939,2023-03-16 00:00:00,374.827820,1.378
40943,2023-03-16 00:04:00,383.092285,1.400
40947,2023-03-16 00:08:00,391.873291,1.420
40951,2023-03-16 00:12:00,366.046844,1.370
40954,2023-03-16 00:15:00,374.311310,1.363
...,...,...,...
134275,2023-05-19 19:36:00,672.864990,1.726
134279,2023-05-19 19:40:00,701.274170,1.695
134283,2023-05-19 19:44:00,771.005676,1.722
134287,2023-05-19 19:48:00,702.823792,1.728


# <font color=yellow>**Parte 2**: Aplicação do algoritmo RF e DecisionTreeRegressor

In [ ]:
#https://pt.stackoverflow.com/questions/428437/como-funciona-o-m%C3%A9todo-train-test-split-no-scikit-learn
#Preparo do dataset para regressão
labels =  ['co_we', 'co_ae']
preffix = ['e2sp_']
label_ref= 'iag_co'

In [43]:
# plt.figure(figsize=(15,5))
# plt.plot(df['Time'], df['SO2'])
# plt.title('Variação de SO2 ao longo do tempo')
# plt.xlabel('Data/Hora')
# plt.ylabel('Concentração (ppb)')
# #plt.grid()
# #plt.show()

In [28]:
#Boxplot 

# plt.figure(figsize=(10,5))
# sns.boxplot(x='day', y='SO2', data=df)
# plt.title('Distribuição de SO2 por dia')
# plt.show()

In [29]:
# #analise do horario

# plt.figure(figsize=(10,5))
# sns.lineplot(x='hour', y='SO2', data=df)
# plt.title('Variação média de SO2 por hora')
# plt.show()

In [30]:
# #compara estatitisticas descritivas entre os dias

# df.groupby('day')['SO2'].describe()

In [32]:
#variação de SO2 por dia e hora

# pivot = df.pivot_table(values='SO2', index='hour', columns='day')
# plt.figure(figsize=(10,5))
# sns.heatmap(pivot, cmap='YlOrRd')
# plt.title('Variação de SO2 por hora e dia')
# plt.show()

In [51]:
from sklearn.tree import DecisionTreeRegressor

# Criando a classe DecisionTreeRegressor()
regressor = DecisionTreeRegressor()

# Ajustando o algoritmo a base de dados
regressor.fit(X_train, y_train)

# Prevendo os resultados
y_pred = regressor.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
# Comparando os valores reais com os valores previstos
df = pd.DataFrame({'Real Values':y_test, 'Predicted Values':y_pred})
df.head()

NameError: name 'y_test' is not defined

In [ ]:
import plotly.express as px

# Plotando gráfico
fig = px.line(df, y=['Real Values', 'Predicted Values'])
fig.show()

In [50]:
#dist cumulativa

# plt.figure(figsize=(10,5))
# sns.ecdfplot(data=df, x='SO2')
# plt.title('Distribuição cumulativa de SO2')
# plt.show()

# <font color=yellow>**Parte 3**: Avaliação de desempenho

Principais métricas para avaliação de desempenho de um modelo de regressão:

* **R² (Coeficiente de Determinação)** - varia de 0 a 1 e representa a proporção da variabilidade na variável dependente que pode ser explicada pelo modelo.
* **MAE (Erro Absoluto Médio)** - avalia a precisão de um modelo prever os dados já observados, sendo a média das diferenças absolutas entre as previsões do modelo e os valores reais.
* **MSE (Erro Quadrático Médio)** - também avalia a precisão do modelo, sendo calculado a partir da média das diferenças ao quadrado entre os valores previstos e os valores reais, penalizando mais erros maiores.
* **RMSE (Raiz do Erro Quadrático Médio)** - é a raiz quadrada do MSE, tornando-se assim uma métrica mais fácil de ser interpretar, já que fica na mesma unidade que o dado original.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calculando R²
r2 = r2_score(y_test, y_pred)

# Calculando MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculando MSE
mse = mean_squared_error(y_test, y_pred)

# Calculando RMSE
rmse = np.sqrt(mse)

print('R2:', r2)
print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)

Um dos principais problemas do algoritmo da Árvore de Decisão é o **OVERFITTING**: 
- Quando na base de treino o modelo tem um bom desempenho, mas na base de teste tem um desempenho ruim
- O modelo se ajustou de forma tão precisa aos dados de treinamento que aprende o ruído aleatório dos dados, mas não generaliza bem para novos dados.



# <font color=yellow>**Parte 4**: Aplicação do algoritmo RandomForest

Parâmetros:
- n_estimators: número de árvores na floresta
- max_features: número máximo de recursos considerados para dividir um nó
- max_depth: número máximo de níveis em cada árvore de decisão
- random_state: semente usada para gerar números aleatórios durante a execução do modelo

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Criando a classe RandomForestRegressor()
rf_regressor = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 18)

# Ajustando o algoritmo a base de dados
rf_regressor.fit(X_train, y_train)

# Prevendo os resultados
y_pred = rf_regressor.predict(X_test)

In [ ]:
# Comparando os valores reais com os valores previstos
df = pd.DataFrame({'Real Values':y_test, 'Predicted Values':y_pred})
df.head()

In [ ]:
# Plotando gráfico
fig = px.line(df, y=['Real Values', 'Predicted Values'])
fig.show()

# <font color=yellow>**Parte 5**: Avaliação de desempenho

In [ ]:
# Calculando R²
r2 = r2_score(y_test, y_pred)

# Calculando MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculando MSE
mse = mean_squared_error(y_test, y_pred)

# Calculando RMSE
rmse = np.sqrt(mse)

print('R2:', r2)
print('MAE:', mae)
print('MSE:', mse)
print('RMSE:', rmse)